# NCEI Paleo Database API

## About

The NCEI Paleo Web Service is searchable via its web interface:[https://www.ncei.noaa.gov/access/paleo-search/](https://www.ncei.noaa.gov/access/paleo-search/) 

Here we are using its API: [https://www.ncei.noaa.gov/access/paleo-search/api](https://www.ncei.noaa.gov/access/paleo-search/api)

Specifically, we are interested in Tree Ring Data




## Tutorial 1: Using Python to access the API for tree ring `.rwl` data

### Step 1: Activate/Create a Python environment

First, you need to import the necessary libraries in Python. The most common ones used for API calls are `requests` for making HTTP requests and `json` for handling JSON data.

We also want to plot the data using `maplotlib`, `numpy`, and `pandas` which are all core libraries for scientific Python.

Later, we will use `fiona` and Leaflet maps to visualize the site locations.

In [ ]:
# import Python libraries
import matplotlib.pyplot as plt
import numpy as np
import json
import requests
import pandas as pd

### Step 2: Set the base url for the API as `api_base`

Define the base url to the NOAA Paleoclimate API, this URL is the starting point for all future requests.

In [ ]:
# Call the NOAA API to get tree ring data 
api_base = "https://www.ncei.noaa.gov/access/paleo-search/study/search.json?"

### Step 3:  Search via API for tree ring (`dataTypeID=18`) studies of raw ring width. 

Returns pages of 10 studies at a time. Set this limit to avoid overwhelming the API with large requests.

Results are returned in JSON format as a dict called "data". 

Note, when searching via the user interface, the API call used to retrieve results is displayed immediately above the search results and can be used to generate a "req_str" relevant to the use case.

In [ ]:

# build the request string for the API 
# data type is tree ring data (18), species is ponderosa pine (PIPO), limit to 10 records
# to limit the results to headersOnly, set headersOnly to true
req_params ="dataPublisher=NOAA&dataTypeId=18&species=PSME&limit=100&headersOnly=false"
req_str = api_base + req_params

# make the request
response = requests.get(req_str)

# Convert the response to JSON
data = response.json()

# Pretty-print the JSON data to debug
# print(json.dumps(data, indent=4))

We want to view the data here, so we use `pandas` to create a DataFrame and print it in MarkDown format



In [ ]:
# Convert the 'study' data to a pandas DataFrame
df = pd.DataFrame(data['study'])

# Convert the DataFrame to a Markdown table
print(df.to_markdown())

## Viewing the API output on a map

### Step 4: install Python library `folium`

We need to install `folium` and its required dependencies into our Python environment. This can be done by using an `environment.yml` (see provided example), or by installing on the fly using `conda`, `mamba`, or `pip`

`mamba install -c conda-forge folium`

### Step 5: Plot the locations of the API results on an interactive map

Create a base folium map, in our example we are zoomed in and centered over North America.

Next, we call the JSON `data` response from the API.

The JSON contains the `geo` geometric coordinates of the study area and its point location

In the map, we are tagging the `doi`, the Site Name and Site ID.

Optionally, if the file has a `.rwl`, a `.pdf`, or `.png` fire history figure associated with it, we provide a hyperlink.

On Mac OS X, press the command button ⌘ and click on the hyperlink to open in your browser. 

If you click the link directly it may open in the Notebook widget. To go back, just re-run the notebook cell

In [147]:
# import packages for Leaflet
import folium

# Create a folium Map centered on the United States
m = folium.Map(location=[37.8, -96], zoom_start=4)

# Assuming that 'data' is your response data
for study in data['study']:
    for site in study['site']:
        lat = float(site['geo']['geometry']['coordinates'][0])
        lon = float(site['geo']['geometry']['coordinates'][1])
        # Create a popup with the site name
        popup_text = f"<a href='{study['doi']}'>url: {study['doi']}</a><br><a>Site Name: </a>{site['siteName']}<br><a>Site ID: </a>{site['NOAASiteId']}<br></a>"
        
        # Add links to any rwl files
        for file in site['paleoData'][0]['dataFile']:
            link = file['fileUrl']
            if link.endswith('.rwl'):
                # click to copy the URL to the clipboard
                popup_text += f"<br><a href='{link}'>view rwl file</a>"
                
        # Add links to any csv files
        for file in site['paleoData'][0]['dataFile']:
            link = file['fileUrl']
            if link.endswith('.csv'):
                popup_text += f"<br><a href='{link}'>Link to csv file</a>"
        
        # Add links to any PDF files
        for file in site['paleoData'][0]['dataFile']:
            link = file['fileUrl']
            if link.endswith('.pdf'):
                popup_text += f"<br><a href='{link}'>Link to PDF file</a>"
        
        # Add links to any figure files
        for file in site['paleoData'][0]['dataFile']:
            link = file['fileUrl']
            if link.endswith('.png'):
                popup_text += f"<br><a href='{link}'>Link to figure file</a>"
        
        popup = folium.Popup(popup_text, max_width=250)
        folium.Marker([lat, lon], popup=popup).add_to(m)

# Display the Map
m

### Saving the Map for later use

Below is a script to save the map as `.html` to your computer 

In [163]:
from datetime import datetime
save_map = input("Do you want to save the map.html locally? (y/n): ")
if save_map == 'y':
    # ask what name to save the map as
    map_name = input("What do you want to name the map? ")
    m.save(f'map_{map_name}.html')
    # if nothing entered
    if map_name == '':
        # Get the current date and time
        now = datetime.now()
        # Format the date and time as a string
        map_name = now.strftime("%Y%m%d_%H%M%S")
        # Use the timestamp in the filename
        m.save(f'map_{map_name}.html')
else:
    print('not saved')

### Step 7: Download the `.rwl` for a selected site using the `siteID`

